# Exercises Day04

In [ ]:
import numpy as np
import scipy 
import matplotlib.pyplot as plt

def compute_confidence_interval_sample(data : list[float], confidence : float = 0.95) -> list[float]:
    """
        Compute the confidence interval of the mean of the data.
        :param data: list of data
        :param confidence: confidence level
        
        :return: confidence interval of the mean of the data
    """
    N = len(data)
    alpha = 1 - confidence

    data = np.array(data)
    #conf_mean = [data.mean - confidence * data.std()/np.sqrt(N), data.mean + confidence * data.std()/np.sqrt(N)]
    conf_mean = [data.mean() + data.std()/np.sqrt(N) * scipy.stats.t.cdf(alpha/2, N - 1), data.mean() + data.std()/np.sqrt(N) * scipy.stats.t.cdf(1-alpha/2, N - 1)]
    return conf_mean


## Ex01

**Estimate an integral $\int_0^1 e^x d x$ by simulation (the Crude Monte Carlo estimator). Use e.g. an estimator based on 100 samples and present the result as the point estimator and a confidence interval.**

In [ ]:
# We assume that we may use numpy because we already went through with sampling from numpy once
# although you didn't explicitely state it as last time...


## Ex02

**Estimate an integral $\int_0^1 e^x d x$ using antithetic variables, with comparable computer ressources.**

## Ex03

**Estimate an integral $\int_0^1 e^x d x$ using a control variable, with comparable computer ressources.**


## Ex04

**Estimate an integral $\int_0^1 e^x d x$ using stratified sampling, with
comparable computer ressources.**


## Ex05

**Use control variates to reduce the variance of the estimator in
exercise 04 (Poisson arrivals).**

## Ex06

**Demonstrate the effect of using common random numbers in exercise 4 for the difference between Poisson arrivals (Part 1) and a renewal process with hyperexponential interarrival times.**

*Remark: You might need to do some thinking and some re-programming* 

## Ex07

## Ex08

## Ex09